# 웹사이트에서 데이터를 가져와서 요구사항에 맞게 가공하는 ETL 파이프라인을 만듭니다.

## 시나리오

- 당신은 해외로 사업을 확장하고자 하는 기업에서 Data Engineer로 일하고 있습니다. 경영진에서 GDP가 높은 국가들을 대상으로 사업성을 평가하려고 합니다.
- 이 자료는 앞으로 경영진에서 지속적으로 요구할 것으로 생각되기 때문에 자동화된 스크립트를 만들어야 합니다.

In [56]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def fetch_gdp_data():
    """
    Wikipedia에서 IMF 데이터를 스크랩하여 국가별 GDP 데이터를 반환합니다.
    """
    url = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 적절한 테이블 선택
    table = soup.find('table', {'class': 'wikitable sortable sticky-header-multi static-row-numbers'})

    if table is None:
        raise ValueError("Unable to find the sortable table on the Wikipedia page.")

    # 테이블 파싱
    rows = table.find_all('tr')
    data = []

    for row in rows[3:]:  
        cols = row.find_all('td')
        if len(cols) < 3:
            continue
            
        country = cols[0].text.strip()
        gdp_text = cols[1].text.strip()
        gdp = re.sub(r'\\[.*?\\]', '', gdp_text).replace(',', '')
        
        try:
            gdp = float(gdp)
        except ValueError:
            continue

        data.append([country, gdp])

    # 데이터프레임 생성
    df = pd.DataFrame(data, columns=['Country', 'GDP (Millions of USD)'])

    # GDP를 1B USD로 변환 및 소수점 2자리까지 표시
    df['GDP (Billions of USD)'] = df['GDP (Millions of USD)'] / 1000
    df = df[['Country', 'GDP (Billions of USD)']]
    df['GDP (Billions of USD)'] = df['GDP (Billions of USD)'].round(2)

    # GDP가 높은 순서대로 정렬
    df_sorted = df.sort_values(by='GDP (Billions of USD)', ascending=False)
    
    return df_sorted

def print_top_countries_by_gdp():
    # 데이터 가져오기
    df_sorted = fetch_gdp_data()
    
    # GDP가 100B USD 이상인 국가 필터링
    df_filtered = df_sorted[df_sorted["GDP (Billions of USD)"] >= 100]

    # GDP가 100B USD 이상인 국가 출력
    print("GDP가 100B USD 이상인 국가들:")
    if not df_filtered.empty:
        print(df_filtered.to_string(index=False))
    else:
        print("No countries with GDP >= 100B USD found.")
    
   # 각 Region별 상위 5개 국가의 GDP 평균 계산
    print("\nTop 5 countries by GDP 평균:")
    top5_avg_gdp = df_sorted.nlargest(5, 'GDP (Billions of USD)')['GDP (Billions of USD)'].mean()
    print(f"Top 5 국가의 GDP 평균: {top5_avg_gdp:.2f} B USD")

# 함수 호출
print_top_countries_by_gdp()

GDP가 100B USD 이상인 국가들:
             Country  GDP (Billions of USD)
       United States               28781.08
               China               18532.63
             Germany                4591.10
               Japan                4110.45
               India                3937.01
      United Kingdom                3495.26
              France                3130.01
              Brazil                2331.39
               Italy                2328.03
              Canada                2242.18
              Russia                2056.84
              Mexico                2017.02
           Australia                1790.35
         South Korea                1760.95
               Spain                1647.11
           Indonesia                1475.69
         Netherlands                1142.51
              Turkey                1113.56
        Saudi Arabia                1106.02
         Switzerland                 938.46
              Poland                 844.62
         

In [71]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import json
import os
from datetime import datetime

# 로그 기록 함수
def log_message(message):
    """
    로그 메시지를 etl_project_log.txt 파일에 기록
    """
    timestamp = datetime.now().strftime('%Y-%B-%d-%H-%M-%S')
    log_entry = f"{timestamp}, {message}\n"
    with open("etl_project_log.txt", "a") as log_file:
        log_file.write(log_entry)

# 데이터 추출 함수
def fetch_gdp_data():
    """
    Wikipedia에서 IMF 데이터를 스크랩하여 국가별 GDP 데이터를 반환합니다.
    """
    log_message("Starting data extraction process") 
    url = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 적절한 테이블 선택
    table = soup.find('table', {'class': 'wikitable sortable sticky-header-multi static-row-numbers'})

    # 테이블이 없을 경우 예외처리
    if table is None:
        log_message("Unable to find the sortable table on the Wikipedia page.")
        raise ValueError("Unable to find the sortable table on the Wikipedia page.")

    # 테이블 모든 행 추출
    rows = table.find_all('tr')
    data = []

    # 각 행의 데이터를 파싱
    for row in rows[3:]:   
        cols = row.find_all('td')
        if len(cols) < 3:
            continue
            
        country = cols[0].text.strip()
        gdp_text = cols[1].text.strip()
        gdp = re.sub(r'\\[.*?\\]', '', gdp_text).replace(',', '')
        
        try:
            gdp = float(gdp)
        except ValueError:
            continue 

        data.append([country, gdp])

    # 추출된 데이터 반환
    log_message("Data extraction process completed")
    return data 

# 데이터 변환 함수
def transform_gdp_data(data):
    """
    추출한 데이터를 변환하여 DataFrame으로 반환합니다.
    """
    log_message("Starting data transformation process")
    
    # 데이터프레임 생성
    df = pd.DataFrame(data, columns=['Country', 'GDP (Millions of USD)'])

    # GDP를 1B USD로 변환 및 소수점 2자리까지 표시
    df['GDP (Billions of USD)'] = df['GDP (Millions of USD)'] / 1000
    df = df[['Country', 'GDP (Billions of USD)']]
    df['GDP (Billions of USD)'] = df['GDP (Billions of USD)'].round(2)

    # GDP가 높은 순서대로 정렬
    df_sorted = df.sort_values(by='GDP (Billions of USD)', ascending=False)
    
    log_message("Data transformation process completed")
    return df_sorted

# 데이터 로드 함수
def load_gdp_data(df, filename='Countries_by_GDP.json'):
    """
    변환된 데이터를 JSON 파일로 저장합니다.
    """
    log_message("Starting data loading process")
    
    # JSON 파일로 저장
    df.to_json(filename, orient='records', lines=True, force_ascii=False)
    
    log_message("Data loading process completed")


# ETL 프로세스 실행 함수
def etl_process():
    """
    ETL 프로세스를 실행합니다.
    """
    log_message("ETL process started")
    
    # 데이터 추출
    data = fetch_gdp_data()
    
    # 데이터 변환
    df_sorted = transform_gdp_data(data)
    
    # 데이터 로드
    load_gdp_data(df_sorted)
    
    # GDP가 100B USD 이상인 국가 출력
    df_filtered = df_sorted[df_sorted["GDP (Billions of USD)"] >= 100]
    print("GDP가 100B USD 이상인 국가들:")
    if not df_filtered.empty:
        print(df_filtered.to_string(index=False))
    else:
        print("No countries with GDP >= 100B USD found.")
    
    # 상위 5개 국가의 GDP 평균 계산
    print("\nTop 5 countries by GDP 평균:")
    top5_avg_gdp = df_sorted.nlargest(5, 'GDP (Billions of USD)')['GDP (Billions of USD)'].mean()
    print(f"Top 5 국가의 GDP 평균: {top5_avg_gdp:.2f} B USD")

    log_message("ETL process completed")

# ETL 프로세스 실행
if __name__ == "__main__":
    etl_process()

GDP가 100B USD 이상인 국가들:
             Country  GDP (Billions of USD)
       United States               28781.08
               China               18532.63
             Germany                4591.10
               Japan                4110.45
               India                3937.01
      United Kingdom                3495.26
              France                3130.01
              Brazil                2331.39
               Italy                2328.03
              Canada                2242.18
              Russia                2056.84
              Mexico                2017.02
           Australia                1790.35
         South Korea                1760.95
               Spain                1647.11
           Indonesia                1475.69
         Netherlands                1142.51
              Turkey                1113.56
        Saudi Arabia                1106.02
         Switzerland                 938.46
              Poland                 844.62
         

In [108]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import json
import os
import sqlite3
from datetime import datetime

# 로그 기록 함수
def log_message(message):
    """
    로그 메시지를 etl_project_log.txt 파일에 기록
    """
    timestamp = datetime.now().strftime('%Y-%B-%d-%H-%M-%S')
    log_entry = f"{timestamp}, {message}\n"
    with open("etl_project_log.txt", "a") as log_file:
        log_file.write(log_entry)

# 데이터 추출 함수
def fetch_gdp_data():
    """
    Wikipedia에서 IMF 데이터를 스크랩하여 국가별 GDP 데이터를 반환합니다.
    """
    log_message("데이터 추출 시작") 
    url = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 적절한 테이블 선택
    table = soup.find('table', {'class': 'wikitable sortable sticky-header-multi static-row-numbers'})

    # 테이블이 없을 경우 예외처리
    if table is None:
        log_message("Wikipedia 페이지에서 테이블을 찾을 수 없음")
        raise ValueError("Wikipedia 페이지에서 테이블을 찾을 수 없음")

    # 테이블 모든 행 추출
    rows = table.find_all('tr')
    data = []

    # 각 행의 데이터를 파싱
    for row in rows[3:]:   
        cols = row.find_all('td')
        if len(cols) < 3:
            continue
            
        country = cols[0].text.strip()
        gdp_text = cols[1].text.strip()
        gdp = re.sub(r'\[.*?\]', '', gdp_text).replace(',', '')
        
        try:
            gdp = float(gdp)
        except ValueError:
            continue 

        data.append([country, gdp])

    # 추출된 데이터 반환
    log_message("데이터 추출 완료")
    return data 

# 데이터 변환 함수
def transform_gdp_data(data):
    """
    추출한 데이터를 변환하여 DataFrame으로 반환합니다.
    """
    log_message("데이터 변환 시작")
    
    # 데이터프레임 생성
    df = pd.DataFrame(data, columns=['Country', 'GDP (Millions of USD)'])

    # GDP를 1B USD로 변환 및 소수점 2자리까지 표시
    df['GDP (Billions of USD)'] = df['GDP (Millions of USD)'] / 1000
    df = df[['Country', 'GDP (Billions of USD)']]
    df['GDP (Billions of USD)'] = df['GDP (Billions of USD)'].round(2)

    # GDP가 높은 순서대로 정렬
    df_sorted = df.sort_values(by='GDP (Billions of USD)', ascending=False)
    
    log_message("데이터 변환 완료")
    return df_sorted

# 데이터 로드 함수 (JSON)
def load_gdp_data_to_json(df, filename='Countries_by_GDP.json'):
    """
    변환된 데이터를 JSON 파일로 저장합니다.
    """
    log_message("데이터를 JSON 파일로 저장 시작")
    
    # JSON 파일로 저장
    df.to_json(filename, orient='records', lines=True, force_ascii=False)
    
    log_message("데이터를 JSON 파일로 저장 완료")

# 데이터 로드 함수 (SQLite)
def load_gdp_data_to_sqlite(df, db_filename='World_Economies.db'):
    """
    변환된 데이터를 SQLite 데이터베이스에 저장합니다.
    """
    log_message("데이터를 SQLite 데이터베이스에 저장 시작")

    # SQLite 데이터베이스에 연결
    conn = sqlite3.connect(db_filename)
    cursor = conn.cursor()
    
    # 테이블 생성
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Countries_by_GDP (
        Country TEXT PRIMARY KEY,
        GDP_USD_billion REAL
    )
    ''')

    # 데이터 삽입
    for _, row in df.iterrows():
        cursor.execute('''
        INSERT OR REPLACE INTO Countries_by_GDP (Country, GDP_USD_billion)
        VALUES (?, ?)
        ''', (row['Country'], row['GDP (Billions of USD)']))
    
    # 변경사항 저장 및 연결 종료
    conn.commit()
    conn.close()
    
    log_message("데이터를 SQLite 데이터베이스에 저장 완료")

# SQL 쿼리 실행 함수
def execute_sql_query(query, db_filename='World_Economies.db'):
    """
    주어진 SQL 쿼리를 실행하고 결과를 반환합니다.
    """
    conn = sqlite3.connect(db_filename)
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

# ETL 프로세스 실행 함수
def etl_process():
    """
    ETL 프로세스를 실행합니다.
    """
    log_message("ETL 프로세스 시작")
    
    # 데이터 추출
    data = fetch_gdp_data()
    
    # 데이터 변환
    df_sorted = transform_gdp_data(data)
    
    # 데이터 로드 (JSON)
    load_gdp_data_to_json(df_sorted)
    
    # 데이터 로드 (SQLite)
    load_gdp_data_to_sqlite(df_sorted)
    
    # SQL 쿼리 실행 및 결과 출력
    print("GDP가 100B USD 이상인 국가들:")
    query = """
    SELECT * 
    FROM Countries_by_GDP 
    WHERE GDP_USD_billion >= 100
    """

    # GDP가 100B USD 이상인 국가 출력
    log_message("GDP가 100B USD 이상인 국가 출력")
    df_filtered = execute_sql_query(query)
    if not df_filtered.empty:
        print(df_filtered.to_string(index=False))
    else:
        print("GDP가 100B USD 이상인 국가가 없습니다.")

    log_message("ETL 프로세스 완료")

# ETL 프로세스 실행
if __name__ == "__main__":
    etl_process()

GDP가 100B USD 이상인 국가들:
             Country  GDP_USD_billion
       United States         28781.08
               China         18532.63
             Germany          4591.10
               Japan          4110.45
               India          3937.01
      United Kingdom          3495.26
              France          3130.01
              Brazil          2331.39
               Italy          2328.03
              Canada          2242.18
              Russia          2056.84
              Mexico          2017.02
           Australia          1790.35
         South Korea          1760.95
               Spain          1647.11
           Indonesia          1475.69
         Netherlands          1142.51
              Turkey          1113.56
        Saudi Arabia          1106.02
         Switzerland           938.46
              Poland           844.62
              Taiwan           802.96
             Belgium           655.19
              Sweden           623.05
           Argentina       